# Download ATLAS/ICESat-2 L3B Gridded Antarctic and Arctic Land Ice Height Change V002

NSICD ATLAS/ICESat-2 L3B Gridded Antarctic and Arctic Land Ice Height V002:
https://cmr.earthdata.nasa.gov/search/concepts/C2500138845-NSIDC_ECS.html<br>


NSICD ATLAS/ICESat-2 L3B Gridded Antarctic and Arctic Land Ice Height Change V002: 

https://cmr.earthdata.nasa.gov/search/concepts/C2500140833-NSIDC_ECS.html  <br><br>



NASA EarthData Search:

https://search.earthdata.nasa.gov/search?q=ATL14+V002 



In [18]:
import itertools
import requests
import os
import IcesheetCHANGES as changes
import collection

## Set options

First, set collections key for desired data set from NASA EarthData and set the region name.
For the collection ID, set collections_key using the dictionary below, or use the following instructions to add a new collection to the dictionary. 

Example: Getting a collection ID from EarthData Search: 
[MEaSUREs Annual Antarctic Ice Velocity Maps V001](https://search.earthdata.nasa.gov/search/granules/collection-details?p=C2245171699-NSIDC_ECS&pg[0][v]=f&pg[0][gsk]=-start_date&q=NSIDC-0720%20V001&tl=1686700071.247!3!!)

While viewing the collection on EarthData, as in the above link, follow "View More Info" to visit the CMR page for the collection.
Then, look for the collections ID in the URL or as a tag below the title. 

![Locating Collection ID from CMR page](getting_collectionID.png)


In [19]:
# Print availible collections
collection.print_collections()

Available collections: 

MEaSUREs Annual Antarctic Ice Velocity Maps V001
Short name: MEaSUREs Antarctic Annual Velocity

MEaSUREs Greenland Quarterly Ice Sheet Velocity Mosaics from SAR and Landsat V005
Short name: MEaSUREs Greenland Quarterly Velocity

MEaSUREs Greenland Monthly Ice Sheet Velocity Mosaics from SAR and Landsat, Version 5
Short name: MEaSUREs Greenland Monthly Velocity

ATLAS/ICESat-2 L3B Gridded Antarctic and Arctic Land Ice Height, Version 2
Short name: ATL14 Antarctic Elevation

ATLAS/ICESat-2 L3B Gridded Antarctic and Arctic Land Ice Height Change, Version 2
Short name: ATL15 Antarctic Elevation



In [20]:
collection_key = 'ATL15 Antarctic Elevation'
region_name = 'Antarctic'
data_type = 'elevation'

collection_id = collection.collection(collection_key)
print('Collection ID: ', collection_id)

Collection ID:  C2500140833-NSIDC_ECS


Next, define two directories on your local drive as follows:

| directory | purpose | 
|-----------|---------|
|`project_folder` | This is the path where output data from the changes module will be stored - the data to be used directly for analysis. | 
|`data_folder` | This is the path where ice velocity and elevation data, from their respective sources, will be stored. The data_folder option was created to facilitate data storage on external drives. |

In [21]:
project_folder = '/Users/tara/Documents/SJSU/MLML/Projects/CHANGES/Examples'
data_folder='/Volumes/Seagate/CHANGES/data_repository/tutorial'

Initialize the AntarcticCHANGES object - this object will contain all pertinent information to initialize the data grids in your region of interest.

In [22]:
AC = changes.AntarcticCHANGES(project_folder, data_folder, collection_key, collection_id, region_name, data_type)

In [23]:
AC.print_attributes()

Region name:		 Antarctic
Data Collection:	 ATLAS/ICESat-2 L3B Gridded Antarctic and Arctic Land Ice Height Change, Version 2
Collection short name:	 ATL15 Antarctic Elevation
Collection ID: 		 C2500140833-NSIDC_ECS
Data type:		 Elevation
Projection:		 3031
Download path:		 /Volumes/Seagate/CHANGES/data_repository/tutorial/Antarctic/Elevation/ATL15 Antarctic Elevation/Data
Metadata path:		 /Users/tara/Documents/SJSU/MLML/Projects/CHANGES/Examples/Antarctic/Elevation/Metadata


## Obtain list of available files for download

In [24]:
# this function is from the ICESat2 data page
def cmr_filter_urls(search_results):
    """Select only the desired data files from CMR response."""
    if 'feed' not in search_results or 'entry' not in search_results['feed']:
        return []
    
    entries = [e['links']
                for e in search_results['feed']['entry']
                if 'links' in e]
    
    # Flatten "entries" to a simple list of links
    links = list(itertools.chain(*entries))

    urls = []
    unique_filenames = set()

    for link in links:
        if 'href' not in link:
            # Exclude links with nothing to download
            continue    # continue jumps to next iteration in the loop
        if 'inherited' in link and link['inherited'] is True:
            # Why are we excluding these links?
            continue
        if 'rel' in link and 'data#' not in link['rel']:
            # Exclude links which are not classified by CMR as "data" or "metadata"
            continue
        if 'title' in link and 'opendap' in link['title'].lower():
            # Exclude OPeNDAP links--they are responsible for many duplicates
            # This is a hack; when the metadata is updated to properly identify
            # non-datapool links, we should be able to do this in a non-hack way
            continue

        filename = link['href'].split('/')[-1]
        
        # Exclude links with duplicate filenames (they would overwrite)
        if filename in unique_filenames:
            continue

        unique_filenames.add(filename)
        urls.append(link['href'])

    return urls

In [25]:
# Build and call the CMR API URL
cmr_query_url = 'https://cmr.earthdata.nasa.gov/search/granules.json?echo_collection_id=' + AC.collection_id + '&page_size=2000'
response = requests.get(cmr_query_url)

print('CMR request URL: ', cmr_query_url)

# print error code based on response
if response.status_code != 200:
    print('ERROR: {}'.format(response.status_code))
search_page = response.json()

# If JSON contains an error message, print the message at the key, 'error'
if 'errors' in search_page:
    print(search_page['errors'])
else: 
    urls = cmr_filter_urls(search_page)
    print("Successfully obtained {} URLs.".format(len(urls)))

CMR request URL:  https://cmr.earthdata.nasa.gov/search/granules.json?echo_collection_id=C2500140833-NSIDC_ECS&page_size=2000
Successfully obtained 56 URLs.


In [26]:
# Get only .nc files and also store the file names in a seperate list
file_links = []
file_names = []
metadata = []

for url in urls:
    if '.nc'in url and not 'xml' in url:
        file_links.append(url)
        url_parts = url.split('/')
        file_name = url_parts[-1]
        file_names.append(file_name)

In [27]:
## write available file names and links to csv file
metadata_folder = AC.metadata_path
csv_filename = AC.short_name + '.csv'

if not os.path.exists(metadata_folder):
    os.makedirs(metadata_folder)

f = open(os.path.join(metadata_folder, csv_filename),'w')
f.write('File_Name,URL')
for ea in range(len(file_names)):
    f.write('\n'+file_names[ea]+','+file_links[ea])
f.close()

print('CSV file written to ' + os.path.join(metadata_folder, csv_filename))

CSV file written to /Users/tara/Documents/SJSU/MLML/Projects/CHANGES/Examples/Antarctic/Elevation/Metadata/ATL15 Antarctic Elevation.csv


In [13]:
# check for existing files, get a list of files not on disk
def obtain_download_list(file_names, file_links):
    
    output_folder = AC.download_path
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    download_list_names=[]
    download_list_links=[]
    for file in file_names:
        download_file = True
        if file not in os.listdir(os.path.join(output_folder)):
            for existing_fil in os.listdir(os.path.join(output_folder)):
                if file == existing_fil:  # this allows for older versions to be kept 
                    download_file = False
        else:
            download_file=False
        if download_file:
            download_list_names.append(file)
            download_list_links.append(file_links[file_names.index(file)])

    print('Number of files to download: ', len(download_list_names))

    return(download_list_names, download_list_links) 

In [14]:
dl_list_names, dl_list_links = obtain_download_list(file_names, file_links)

Number of files to download:  1


## Download the data

In [18]:
def download_nc_direct(file_names, file_links):
    if not os.path.exists(AC.download_path):
        os.makedirs(AC.download_path)
    
    not_downloaded_names = []
    not_downloaded_links = []

    print('Downloading ' + str(len(file_names)) + ' file(s)...')
    for i in range(len(file_names)):

        print('Downloading ' + str(i + 1) + '/' + str(len(file_names)) + ': ' + file_names[i], end='\r')

        # get file
        r = requests.get(file_links[i], allow_redirects=True)
        if r.status_code != 200:    # 200 is the standard response for successful HTTP requests
            print('ERROR: ' + str(r.status_code) + ': ' + file_links[i] + '\n')
            not_downloaded_names.append(file_names[i])
            not_downloaded_links.append(file_links[i])
        # write content to file
        open(os.path.join(AC.download_path,file_names[i]), 'wb').write(r.content)
    return(not_downloaded_names, not_downloaded_links)

In [19]:
def run_download_nc_direct(dl_list_names, dl_list_links):
    # Download the files
    not_downloaded_names, not_downloaded_links = download_nc_direct(dl_list_names, dl_list_links)

    # Attempt to download any files that were not downloaded the first time
    if len(not_downloaded_names) > 0:
        print('The following files were not downloaded:')
        for i in range(len(not_downloaded_names)):
            print(not_downloaded_names[i] + ': ' + not_downloaded_links[i])
            
        print('Attempting to download these files once more...')
        not_downloaded_names, not_downloaded_links = download_nc_direct(not_downloaded_names, not_downloaded_links)
        if len(not_downloaded_names) > 0:
            print('Please download these files manually and place them in the following folder:')
            print(AC.download_path)

In [20]:
run_download_nc_direct(dl_list_names, dl_list_links)